<a href="https://colab.research.google.com/github/iterat1on/class/blob/main/test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
!pip install tensorflow_addons
!pip install opencv-contrib-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-tet3c1kz
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-tet3c1kz
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"iterat1on","key":"fd9b26ce3c2bf39c4544405767fb1f48"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d ibombonato/xray-body-images-in-png-unifesp-competion

 95% 190M/200M [00:01<00:00, 125MB/s]
100% 200M/200M [00:01<00:00, 116MB/s]


In [12]:
! unzip -qq xray-body-images-in-png-unifesp-competion.zip

In [13]:
import os
# os.remove('xray-body-images-in-png-unifesp-competion.zip')

In [14]:
os.listdir()

['.config',
 'test_df.csv',
 'sample_submission.csv',
 'train_df.csv',
 'images',
 'xray-body-images-in-png-unifesp-competion.zip',
 'image_png.png',
 'kaggle.json',
 'sample_data']

In [15]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import albumentations as album
import matplotlib.pyplot as plt
from skimage.io import imread
from sklearn.model_selection import StratifiedKFold

AttributeError: ignored

In [9]:
# !pip uninstall opencv-contrib-python

Found existing installation: opencv-contrib-python 4.1.2.30
Uninstalling opencv-contrib-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_contrib_python-4.1.2.30.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/cv2/config-3.py
    /usr/local/lib/python3.7/dist-packages/cv2/config.py
    /usr/local/lib/python3.7/dist-packages/cv2/cv2.abi3.so
    /usr/local/lib/python3.7/dist-packages/cv2/gapi/__init__.py
    /usr/local/lib/python3.7/dist-packages/cv2/load_config_py2.py
    /usr/local/lib/python3.7/dist-packages/cv2/load_config_py3.py
    /usr/local/lib/python3.7/dist-packages/cv2/mat_wrapper/__init__.py
    /usr/local/lib/python3.7/dist-packages/cv2/misc/__init__.py
    /usr/local/lib/python3.7/dist-packages/cv2/misc/version.py
    /usr/local/lib/python3.7/dist-packages/cv2/utils/__init__.py
    /usr/local/lib/python3.7/dist-packages/cv2/version.py
Procee

In [12]:
N_SPLITS = 3
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 16
EPOCHS = 200
PATIENCE = 8
MAX_LR = 1e-5
MIN_LR = 1e-6

In [13]:
class_names = np.array(["Abdomen", "Ankle", "Cervical Spine", "Chest", 
                        "Clavicles", "Elbow", "Feet", "Finger",
                        "Forearm", "Hand", "Hip", "Knee",  
                        "Lower Leg", "Lumbar Spine", "Others", "Pelvis", 
                        "Shoulder", "Sinus", "Skull", "Thigh", 
                        "Thoracic Spine", "Wrist"])

In [14]:
df = pd.read_csv("/content/train.csv")
df.head()

,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10025629581362719970...,0
1,1.2.826.0.1.3680043.8.498.10036150326276641158...,15
2,1.2.826.0.1.3680043.8.498.10038426859954986240...,12
3,1.2.826.0.1.3680043.8.498.10050991192143676483...,14
4,1.2.826.0.1.3680043.8.498.10053309524595490852...,3


In [15]:
images_path = "/content/train/"
temp_SOPInstanceUID = []
for i in df.SOPInstanceUID:
    temp_SOPInstanceUID.append(images_path + i + ".png")
df.SOPInstanceUID = temp_SOPInstanceUID

In [16]:
num_classes = 22
targets = np.zeros((len(df.Target), num_classes), dtype=int)
for i, t in enumerate(df.Target):
    target = t.strip().split()
    for j in target:
        targets[i, int(j)] = 1

In [17]:
df2 = pd.DataFrame(targets, columns=class_names)
df = pd.concat([df, df2], axis=1) 
df.head()

,SOPInstanceUID,Target,Abdomen,Ankle,Cervical Spine,Chest,Clavicles,Elbow,Feet,Finger,...,Lower Leg,Lumbar Spine,Others,Pelvis,Shoulder,Sinus,Skull,Thigh,Thoracic Spine,Wrist
0,/content/train/1.2.826.0.1.3680043.8.498.10025...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,/content/train/1.2.826.0.1.3680043.8.498.10036...,15,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,/content/train/1.2.826.0.1.3680043.8.498.10038...,12,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,/content/train/1.2.826.0.1.3680043.8.498.10050...,14,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,/content/train/1.2.826.0.1.3680043.8.498.10053...,3,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
